In [21]:
import json
import re
import core.utils as oa
from rapidfuzz import fuzz
import pandas as pd
import statistics
import io
import os

import datetime
from numpyencoder import NumpyEncoder
from pathlib import Path

# root directory path
ROOT = Path(os.getcwd()).resolve().parents[0]

In [22]:
date = datetime.datetime.now().strftime("%y-%m-%d_%H:%M")
date

'25-09-19_12:38'

In [24]:
print("Sollen Bibelstellen (1) oder Liedzitate (2) gesucht werden?")
response = None
while response not in ["1", "2"]:
    response = input("Bitte Nummer eingeben")

if response == "1":
    task = "bibel"
    print("Bibelstellen werden gesucht")
else:
    task = "lieder"
    print("Liedzitate werden gesucht")


Sollen Bibelstellen (1) oder Liedzitate (2) gesucht werden?
Bibelstellen werden gesucht


In [25]:
print("Sollen Predigten mit den meisten (1), oder den längsten (2) Liedzitaten analysiert werden?")
response = None
while response not in ["1", "2"]:
    response = input("Bitte Nummer eingeben")

if response == "2":
    corpus = "longest"
else:
    corpus = "most"


with open(ROOT / f"sermons_with_{corpus}_music.json", "r") as f:
    testsermons = json.load(f)

Sollen Predigten mit den meisten (1), oder den längsten (2) Liedzitaten analysiert werden?


In [28]:
relevant_page_texts = []
if task == "lieder":
    for n in range(41, 1291):
        with open(ROOT / f"source_texts/praxis_pietatis_verses/{n}.json") as f:
            page = json.load(f)
        page_info = {}
        page_info[n] = page
        relevant_page_texts.append(page_info)
else:
    with open(ROOT / "source_texts/bible/old_testament_chunked.json") as f:
        old_testament = json.load(f)
    for book_name, chapters in old_testament.items():
        for chapter, verses in chapters.items():
            for verse_nr, verse in verses.items():
                verse_info = {}
                verse_id = f"{book_name}_{chapter}_{verse_nr}"
                verse_info[verse_id] = verse
                relevant_page_texts.append(verse_info)
    with open(ROOT / f"source_texts/bible/new_testament_chunked.json") as f:
        new_testament = json.load(f)
    for book_name, chapters in old_testament.items():
        for chapter, verses in chapters.items():
            for verse_nr, verse in verses.items():
                verse_info = {}
                verse_id = f"{book_name}_{chapter}_{verse_nr}"
                verse_info[verse_id] = verse
                relevant_page_texts.append(verse_info)

In [29]:
fuzziness = 90

In [30]:
similarity_table = {}
similarity_table["task"] = task
similarity_table['date'] = date
similarity_table['corpus'] = corpus
similarity_table['method'] = 'similarity_search'
similarity_table['fuzziness'] = fuzziness


In [31]:
testsermons = ["E000036"]

In [32]:
def remove_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    """Removes duplicate matches for sentences in quote classification

    Args:
        df (pd.DataFrame): The Dataframe containing the sentence matches

    Returns:
        pd.DataFrame: The Dataframe with duplicates removed
    """
    def find_duplicate_satz(df):
        duplicate_indices = {}
        for par_satz, indices in df.groupby(['Paragraph', 'Satz']).groups.items():
            if len(indices) > 1:
                satz_id = str(par_satz[0]) +  "-" + str(par_satz[1])
                duplicate_indices[satz_id] = list(indices)  # Convert indices to a list
        return duplicate_indices

    for satz_id, indices in find_duplicate_satz(df.copy()).items():
        satz = [int(x) for x in satz_id.split("-")]
        if indices[0]-1 in df.index:
            check = df["Bibelstelle"][indices[0]-1]
        else:
            check = df["Bibelstelle"][indices[-1]+1]
        print(check)
        matches = df.query(f"Paragraph == {satz[0]} and Satz == {satz[1]} and Bibelstelle == '{check}'").index
        if len(matches):
            match_index = matches[0]
            for i in indices:
                if i != match_index:
                    df.drop([i], inplace=True)

    df.sort_values(by=["Ähnlichkeit"], inplace=True)
    df.drop_duplicates(subset=['Paragraph','Satz'], keep='last', inplace=True)
    df.sort_index(inplace=True)
    df.reset_index(drop=True)

    return df

In [33]:
similarity_table['results'] = []
if task == "lieder":
    for id in testsermons:
        print(f"Starting with {id}")
        sermon = oa.Sermon(id)

        # perform classification
        hits = []
        all_sents = 0
        for i in range(len(sermon.chunked)):                # for each paragraph
            for j in range(len(sermon.chunked[i])):         # for each sentence
                if " bibel" in sermon.chunked[i][j]["types"]:
                    continue
                else:
                    all_sents += 1
                    query = " ".join(sermon.chunked[i][j]["words"])
                    query = re.sub(r'[/.,;:?!]', '', query)
                    for page in relevant_page_texts:
                        for pagenr, verses in page.items():
                            for verse in verses:
                                sim_score = fuzz.ratio(query, verse)
                                if sim_score >= fuzziness:
                                    hits.append([query, i, j, pagenr, verse, float(f"{sim_score:.2f}")])

        guessed_hits = pd.DataFrame(hits, columns=["Predigt", "Paragraph", "Satz", "Liederbuch", "Liedvers", "Ähnlichkeit"])     # create dataframe
        guessed_hits['Dopplung'] = guessed_hits.groupby('Satz')['Satz'].transform(lambda x: x.duplicated())

        similarity_table['results'].append([id, all_sents, guessed_hits.to_csv()[1:]])
else:
    for id in testsermons:
        print(f"Starting with {id}")
        sermon = oa.Sermon(id)

        # perform classification
        hits = []
        all_sents = 0
        for i in range(len(sermon.chunked)):                # for each paragraph
            for j in range(len(sermon.chunked[i])):         # for each sentence
                if " musikwerk" in sermon.chunked[i][j]["types"]:
                    continue
                else:
                    all_sents += 1
                    query = " ".join(sermon.chunked[i][j]["words"])
                    query = re.sub(r'[/.,;:?!]', '', query)
                    for page in relevant_page_texts:
                        for pagenr, verses in page.items():
                            for verse in verses:
                                sim_score = fuzz.ratio(query, verse)
                                if sim_score >= fuzziness:
                                    hits.append([query, i, j, pagenr, verse, float(f"{sim_score:.2f}")])

        guessed_hits = pd.DataFrame(hits, columns=["Predigt", "Paragraph", "Satz", "Bibelstelle", "Bibelvers", "Ähnlichkeit"])     # create dataframe
        guessed_hits['Dopplung'] = guessed_hits.groupby('Satz')['Satz'].transform(lambda x: x.duplicated())

        similarity_table['results'].append([id, all_sents, guessed_hits.to_csv()[1:]])

Starting with E000036


In [34]:
guessed_hits = remove_duplicates(guessed_hits).reset_index(drop=True)

Psalm_38_9
Psalm_73_25
Psalm_38_9
Psalm_116_7
Psalm_37_4
Jesaja_52_7
Psalm_64_10
Psalm_100_4
Psalm_64_10
Psalm_100_4
Psalm_147_1
2 Könige_13_21
Psalm_147_1
Hesekiel_41_20
Amos_5_23
Hohelied_6_3
Hohelied_7_11
Psalm_73_23
Psalm_73_24
1 Könige_18_15
Psalm_16_11
2 Könige_5_16


In [39]:
guessed_hits

,Predigt,Paragraph,Satz,Bibelstelle,Bibelvers,Ähnlichkeit,Dopplung
0,so frage ich nichts nach himmel und erden,21,15,Psalm_73_25,so frage ich nichts nach Himel vnd Erden,91.36,True
1,es ist mit mir gar anders,23,27,Psalm_38_9,Es ist mit mir gar anders,96.00,False
2,sey nun wieder zufrieden meine seele,23,40,Psalm_116_7,Sey nu wider zu frieden meine Seele,90.14,True
3,der wird dir geben was dein hertz wünschet,23,47,Psalm_37_4,Der wird dir geben was dein hertz wündschet,96.47,False
4,die da friede verkündigen,27,8,Jesaja_52_7,die da Friede verkündigen,96.00,True
5,die da sagen zu zion,27,10,Jesaja_52_7,Die da sagen zu Zion,90.00,False
6,werden sagen das hat gott gethan,29,1,Psalm_64_10,werden sagen Das hat Gott gethan,93.75,False
7,und mercken daß es sein werck sey,29,2,Psalm_64_10,vnd mercken das sein werck sey,92.06,True
8,gehet zu seinen thoren ein mit dancken,29,13,Psalm_100_4,Gehet zu seinen Thoren ein mit dancken,94.74,False
9,zu seinen vorhöfen mit loben,29,14,Psalm_100_4,zu seinen Vorhöfen mit loben,96.43,False


In [36]:
similarity_table

{'task': 'bibel',
 'date': '25-09-19_12:38',
 'corpus': 'most',
 'method': 'similarity_search',
 'fuzziness': 90,
 'results': [['E000036',
   1733,
   'Predigt,Paragraph,Satz,Bibelstelle,Bibelvers,Ähnlichkeit,Dopplung\n0,so frage ich nichts nach himmel und erden,21,15,Psalm_73_25,so frage ich nichts nach Himel vnd Erden,91.36,False\n1,so frage ich nichts nach himmel und erden,21,15,Psalm_73_25,so frage ich nichts nach Himel vnd Erden,91.36,True\n2,es ist mit mir gar anders,23,27,Psalm_38_9,Es ist mit mir gar anders,96.0,False\n3,es ist mit mir gar anders,23,27,Psalm_38_9,Es ist mit mir gar anders,96.0,True\n4,sey nun wieder zufrieden meine seele,23,40,Psalm_116_7,Sey nu wider zu frieden meine Seele,90.14,False\n5,sey nun wieder zufrieden meine seele,23,40,Psalm_116_7,Sey nu wider zu frieden meine Seele,90.14,True\n6,der wird dir geben was dein hertz wünschet,23,47,Psalm_37_4,Der wird dir geben was dein hertz wündschet,96.47,False\n7,der wird dir geben was dein hertz wünschet,23,47,Psal

In [ ]:
with open(ROOT / f'similarity_tables/sim-search_{task}_{corpus}_{fuzziness}_{date}.json', "w") as f:
    json.dump(similarity_table, f, ensure_ascii=False)

In [38]:
buffer = io.StringIO(similarity_table['results'][0][2])
df2 = pd.read_csv(buffer)
df2

,Predigt,Paragraph,Satz,Bibelstelle,Bibelvers,Ähnlichkeit,Dopplung
0,so frage ich nichts nach himmel und erden,21,15,Psalm_73_25,so frage ich nichts nach Himel vnd Erden,91.36,False
1,so frage ich nichts nach himmel und erden,21,15,Psalm_73_25,so frage ich nichts nach Himel vnd Erden,91.36,True
2,es ist mit mir gar anders,23,27,Psalm_38_9,Es ist mit mir gar anders,96.00,False
3,es ist mit mir gar anders,23,27,Psalm_38_9,Es ist mit mir gar anders,96.00,True
4,sey nun wieder zufrieden meine seele,23,40,Psalm_116_7,Sey nu wider zu frieden meine Seele,90.14,False
5,sey nun wieder zufrieden meine seele,23,40,Psalm_116_7,Sey nu wider zu frieden meine Seele,90.14,True
6,der wird dir geben was dein hertz wünschet,23,47,Psalm_37_4,Der wird dir geben was dein hertz wündschet,96.47,False
7,der wird dir geben was dein hertz wünschet,23,47,Psalm_37_4,Der wird dir geben was dein hertz wündschet,96.47,True
8,die da friede verkündigen,27,8,Jesaja_52_7,die da Friede verkündigen,96.00,False
9,die da friede verkündigen,27,8,Jesaja_52_7,die da Friede verkündigen,96.00,True
